In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import keras.layers as lay
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [36]:
class fade_in(keras.Layer):
    def __init__(self, alpha=0.0, activity_regularizer=None, trainable=True, dtype=None, autocast=True, name=None, **kwargs):
        super().__init__(activity_regularizer=activity_regularizer, trainable=trainable, dtype=dtype, autocast=autocast, name=name, **kwargs)
        self.alpha = tf.Variable(alpha,trainable=False)

    def call(self,inputs):
        antigo, novo = inputs
        return self.alpha*novo + (1-self.alpha)*antigo
        

def g_block(x, filters):
    x = lay.UpSampling2D((2,2))(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same')(x)
    x = lay.LeakyReLU(.2)(x)
    return x


def build_generator_4x4(latent_dim=128):
    inputs = lay.Input((latent_dim,))
    x = inputs
    x = lay.Dense(4*4*512,activation='leaky_relu')(x)
    x = lay.Reshape((4,4,512),name='lastfeat_4x4')(x)
    img_output = lay.Conv2D(3,(1,1),(1,1),'same',activation='sigmoid',name='toRGB_4x4')(x)
    return keras.Model(inputs,img_output,name='Generator_4x4')

def build_generator_8x8(g_4x4):
    latent_input = g_4x4.input
    g_4x4.trainable = False
    last_featuremap = g_4x4.get_layer('lastfeat_4x4').output
    old_path = lay.UpSampling2D((2,2))(g_4x4.output)
    x = g_block(last_featuremap,512)
    new_path = lay.Conv2D(3,(1,1),(1,1),'same',activation='sigmoid',name='toRGB_8x8')(x)
    output_fade_in = fade_in()([old_path,new_path])
    return keras.Model(latent_input,output_fade_in,name='Generator_8x8')

def build_generator_16x16(g_8x8):
    latent_input = g_8x8.input
    g_8x8.trainable = False
    last_featuremap = g_8x8.layers[-4].output
    old_path = lay.UpSampling2D((2,2))(g_8x8.output)
    x = g_block(last_featuremap,512)
    new_path = lay.Conv2D(3,(1,1),(1,1),'same',activation='sigmoid',name='toRGB_16x16')(x)
    output_fade_in = fade_in()([old_path,new_path])
    return keras.Model(latent_input,output_fade_in,name='Generator_16x16')

def build_generator_32x32(g_16x16):
    latent_input = g_16x16.input
    g_16x16.trainable = False
    last_featuremap = g_16x16.layers[-4].output
    old_path = lay.UpSampling2D((2,2))(g_16x16.output)
    x = g_block(last_featuremap,512)
    new_path = lay.Conv2D(3,(1,1),(1,1),'same',activation='sigmoid',name='toRGB_32x32')(x)
    output_fade_in = fade_in()([old_path,new_path])
    return keras.Model(latent_input,output_fade_in,name='Generator_32x32')

In [ ]:
def build_discriminator_4x4(filters=128):
    inputs = lay.Input((4,4,3))
    x = inputs
    x = lay.Conv2D(512,(3,3),(1,1),'same',activation='leaky_relu',name='fromRGB_4x4')(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same',activation='leaky_relu')(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same',activation='leaky_relu')(x)
    x = lay.Flatten()(x)
    x = lay.Dense(1,activation='sigmoid')(x)
    return keras.Model(inputs,x)

def build_discriminator_8x8(d_4x4, filters=128):
    inputs = lay.Input((8,8,3))
    d_4x4.trainable = False
    


In [43]:
discriminator = build_discriminator_4x4(filters=512)
discriminator.summary()

Model: "functional_44"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 4, 4, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fromRGB_4x4 (Conv2D)            │ (None, 4, 4, 512)      │        14,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,742,145 (18.09 MB)

 Trainable params: 4,742,145 (18.09 MB)

 Non-trainable params: 0 (0.00 B)